In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

/Users/pradyumna/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# 3.1. Individual GLAM estimation and out of sample prediction

## Load data

In [4]:
# Load data
sufix = '_individual_More_NoBin_Gamma-11_NUTS_33'
data = pd.read_csv('data/PF2019_data/GlamDataPF2019_More_NoBin_33.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,1,0,1,1734.284,110,131,0.669090,0.330910
1,1,1,0,6555.370,47,50,0.759630,0.240370
2,1,2,0,3174.566,50,44,0.549371,0.450629
3,1,3,1,2877.579,57,50,0.608409,0.391591
4,1,4,1,1806.310,42,50,0.522849,0.477151


## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

#test_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_test'+sufix+'.csv'))
#train_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1920 trials) and test (1920 trials) sets...


In [6]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,1,0,1,1734.284,110,131,0.669090,0.330910
2,1,2,0,3174.566,50,44,0.549371,0.450629
4,1,4,1,1806.310,42,50,0.522849,0.477151
6,1,6,1,3650.266,78,80,0.682034,0.317966
8,1,8,1,1259.268,50,48,0.508019,0.491981
10,1,10,1,5698.027,46,50,0.582089,0.417911
12,1,12,1,2626.168,50,56,0.666353,0.333647
14,1,14,1,1671.149,110,124,0.508035,0.491965
16,1,16,1,1527.826,64,80,0.473512,0.526488
18,1,18,1,2267.665,101,110,0.553242,0.446758


## Hierarchical GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_PF2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
  Found old parameter estimates in "results/estimates". Skipping estimation...


In [8]:
# Save parameter estimates
#np.save(str('results/estimates/glam_PF2019_full'+sufix+'.npy'), glam_full.estimates)

In [9]:
pd.DataFrame.from_dict(glam_full.estimates)

,0
0,"{'tau': 0.28, 'gamma': 0.85, 's': 0.009766, 'S..."
1,"{'tau': 0.29, 'gamma': 0.89, 's': 0.005238, 'S..."
2,"{'tau': 0.13, 'gamma': -0.09, 's': 0.007449, '..."
3,"{'tau': 0.17, 'gamma': 0.98, 's': 0.010301, 'S..."
4,"{'tau': 0.13, 'gamma': 0.68, 's': 0.005436, 'S..."
5,"{'tau': 0.08, 'gamma': 0.63, 's': 0.007799, 'S..."
6,"{'tau': 0.1, 'gamma': 0.82, 's': 0.011048, 'SN..."
7,"{'tau': 0.06, 'gamma': -0.27, 's': 0.007531, '..."
8,"{'tau': 0.02, 'gamma': 0.99, 's': 0.005685, 'S..."
9,"{'tau': 0.32, 'gamma': 0.08, 's': 0.009549, 'S..."


In [10]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_PF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_PF2019_full'+ sufix +'.npy'))

# Compute WAICs
#np.save(str('results/waic/glam_PF2019_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...
  Found old DIC scores in "results/waic". Skipping WAIC computation...


In [11]:
glam_full.waic

array([[1.04604884e+03, 0.00000000e+00, 1.72250244e+00, 1.00000000e+00],
       [1.22067883e+03, 0.00000000e+00, 1.77895232e+00, 1.00000000e+00],
       [1.08131950e+03, 0.00000000e+00, 4.57825376e+00, 1.00000000e+00],
       [1.19129127e+03, 0.00000000e+00, 1.81216772e+00, 1.00000000e+00],
       [1.09360688e+03, 0.00000000e+00, 2.09341500e+00, 1.00000000e+00],
       [1.07141022e+03, 0.00000000e+00, 3.96698860e+00, 1.00000000e+00],
       [1.16532728e+03, 0.00000000e+00, 1.59540281e+00, 1.00000000e+00],
       [1.04997833e+03, 0.00000000e+00, 2.34431071e+00, 1.00000000e+00],
       [9.92213429e+02, 0.00000000e+00, 2.57769666e+00, 1.00000000e+00],
       [1.03406225e+03, 0.00000000e+00, 2.59437231e+00, 1.00000000e+00],
       [1.20205192e+03, 0.00000000e+00, 3.10274965e+00, 1.00000000e+00],
       [1.00093682e+03, 0.00000000e+00, 2.61991370e+00, 1.00000000e+00],
       [1.05702793e+03, 0.00000000e+00, 1.21914822e+01, 1.00000000e+00],
       [1.07529609e+03, 0.00000000e+00, 3.57531961e

In [12]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
#np.save(str('results/loo/glam_PF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'GLAM' object has no attribute 'trace'

In [13]:
glam_full.loo

AttributeError: 'GLAM' object has no attribute 'loo'

In [14]:
glam_full.type = "individual"

In [15]:
glam_full.error_range

AttributeError: 'GLAM' object has no attribute 'error_range'

In [16]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_PF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_PF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1920 trials) with new data (1920 trials)...


ValueError: Domain error in arguments.

### 1. no-bias GLAM

In [ ]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM individually...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy'))
 

In [ ]:
   
# Save parameter estimates
#np.save(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

In [ ]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

In [ ]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

#np.save(str('results/loo/glam_PF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

In [ ]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
#    glam_nobias.prediction.to_csv(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv'))

glam_nobias.prediction.head()

## 2. Plot fit

In [11]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

Close Figure to continue...


AttributeError: 'GLAM' object has no attribute 'prediction'

## Parameters for full hierarchical model

In [24]:
params_participant = glam_full.estimates
params_participant

[{'b': 1.0,
  'p_error': 0.05,
  'v': 5.1e-05,
  'gamma': -0.77,
  'SNR': 125.41,
  's': 0.0081,
  'tau': 0.01,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 1.6e-05,
  'gamma': -0.99,
  'SNR': 290.78,
  's': 0.006238,
  'tau': 0.05,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3.5e-05,
  'gamma': -0.11,
  'SNR': 230.96,
  's': 0.008048,
  'tau': 0.11,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 2.1e-05,
  'gamma': -0.99,
  'SNR': 434.59,
  's': 0.009583,
  'tau': 0.1,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3.2e-05,
  'gamma': -0.99,
  'SNR': 165.04,
  's': 0.005638,
  'tau': 0.06,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 5.7e-05,
  'gamma': -0.71,
  'SNR': 83.7,
  's': 0.005543,
  'tau': 0.09,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3e-05,
  'gamma': -0.99,
  'SNR': 232.79,
  's': 0.008216,
  'tau': 0.08,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 5.1e-05,
  'gamma

In [21]:
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

AttributeError: 'list' object has no attribute 'item'

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 